### settings

In [1]:
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
import nipype.interfaces.fsl.maths as fsl
import pandas as pd
import os, re, json

In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
binar = pe.Workflow(name="binarise") # workflow to run the analysis

### data grabbing and sinking

In [3]:
dg = pe.Node(DataGrabber(infields=['sub', 'TR', 'data_folder'], 
                         outfields=['ts']), name='data-grabber')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'ts': [['data_folder', 'sub', 'TR']]}
dg.inputs.field_template = {'ts': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/%s/sub-%s/TR%s/THRSH/spmT*.nii.gz'}

In [4]:
# test it
dg.inputs.sub = '01'
dg.inputs.TR = '700'
dg.inputs.data_folder = 'glmTDFAST_TC'
res = dg.run()
res.outputs

210522-20:57:50,632 nipype.workflow INFO:
	 [Node] Setting-up "data-grabber" in "/tmp/tmpi1m2pgh6/data-grabber".
210522-20:57:50,638 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210522-20:57:50,645 nipype.workflow INFO:
	 [Node] Finished "data-grabber".



ts = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/THRSH/spmT_0004_thresh.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/THRSH/spmT_0005_thresh.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/THRSH/spmT_0006_thresh.nii.gz']

In [5]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath[0])
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0], [t for t in l if re.search('TR([0-9])', t)][0]
    name = '/'.join(name)
    name
    return name

In [6]:
# directory requires setting!
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_apply-bin([0-9]*)', '')] 
ds.inputs.regexp_substitutions = substitutions

In [7]:
# to feed into datagrabber
datinf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                 name='dat-info')
datinf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
#datinf.iterables = [('sub', ['04']), ('TR', ['700', '1510', '1920'])]
datinf.iterables 

[('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]

### threshold

In [8]:
applybin = pe.MapNode(fsl.UnaryMaths(), iterfield = ['in_file'], name='apply-bin')
applybin.inputs.operation = 'bin'
applybin.inputs.args = '-thr .01'
applybin.inputs.nan2zeros = True

### link workflow

In [9]:
binar.connect([(datinf, dg, [('sub', 'sub')]),
               (datinf, dg, [('TR', 'TR')]),
               (dg, ds, [(('ts', printSubPath), 'container')]),
               (dg, applybin, [('ts', 'in_file')]), 
               (applybin, ds, [('out_file', 'MASK')])
])
res = binar.run()

210522-20:57:53,734 nipype.workflow INFO:
	 Workflow binarise settings: ['check', 'execution', 'logging', 'monitoring']
210522-20:57:53,794 nipype.workflow INFO:
	 Running serially.
210522-20:57:53,795 nipype.workflow INFO:
	 [Node] Setting-up "binarise.data-grabber" in "/tmp/tmpi1m2pgh6/data-grabber".
210522-20:57:53,800 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210522-20:57:53,805 nipype.workflow INFO:
	 [Node] Finished "binarise.data-grabber".
210522-20:57:53,806 nipype.workflow INFO:
	 [Node] Setting-up "binarise.apply-bin" in "/tmp/tmp5opn7asf/binarise/_TR_1920_sub_05/apply-bin".
210522-20:57:53,812 nipype.workflow INFO:
	 [Node] Setting-up "_apply-bin0" in "/tmp/tmp5opn7asf/binarise/_TR_1920_sub_05/apply-bin/mapflow/_apply-bin0".
210522-20:57:53,816 nipype.workflow INFO:
	 [Node] Running "_apply-bin0" ("nipype.interfaces.fsl.maths.UnaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives

210522-20:57:55,152 nipype.workflow INFO:
	 [Node] Finished "_apply-bin0".
210522-20:57:55,154 nipype.workflow INFO:
	 [Node] Setting-up "_apply-bin1" in "/tmp/tmp85f814zh/binarise/_TR_700_sub_05/apply-bin/mapflow/_apply-bin1".
210522-20:57:55,158 nipype.workflow INFO:
	 [Node] Running "_apply-bin1" ("nipype.interfaces.fsl.maths.UnaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-05/TR700/THRSH/spmT_0005_thresh.nii.gz -nan -bin -thr .01 /tmp/tmp85f814zh/binarise/_TR_700_sub_05/apply-bin/mapflow/_apply-bin1/spmT_0005_thresh_bin.nii.gz
210522-20:57:55,296 nipype.workflow INFO:
	 [Node] Finished "_apply-bin1".
210522-20:57:55,298 nipype.workflow INFO:
	 [Node] Setting-up "_apply-bin2" in "/tmp/tmp85f814zh/binarise/_TR_700_sub_05/apply-bin/mapflow/_apply-bin2".
210522-20:57:55,303 nipype.workflow INFO:
	 [Node] Running "_apply-bin2" ("nipype.interfaces.fsl.maths.UnaryMaths"), a CommandLine Interface with command:
fslmaths 

210522-20:57:56,461 nipype.workflow INFO:
	 [Node] Finished "_apply-bin2".
210522-20:57:56,464 nipype.workflow INFO:
	 [Node] Finished "binarise.apply-bin".
210522-20:57:56,464 nipype.workflow INFO:
	 [Node] Setting-up "binarise.sink-stuff" in "/tmp/tmpb50lmy0l/binarise/_TR_1510_sub_04/sink-stuff".
210522-20:57:56,471 nipype.workflow INFO:
	 [Node] Running "sink-stuff" ("nipype.interfaces.io.DataSink")
210522-20:57:56,472 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-04/TR1510/MASK/_TR_1510_sub_04/_apply-bin0/spmT_0004_thresh_bin.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-04/TR1510/MASK///spmT_0004_thresh_bin.nii.gz
210522-20:57:56,475 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-04/TR1510/MASK/_TR_1510_sub_04/_apply-bin1/spmT_0005_thresh_bin.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-04/TR1510/MASK///spmT_0005_thresh_bin.nii.gz
210522-20:57:56,476 

210522-20:57:57,531 nipype.workflow INFO:
	 [Node] Finished "binarise.sink-stuff".
210522-20:57:57,531 nipype.workflow INFO:
	 [Node] Setting-up "binarise.data-grabber" in "/tmp/tmpi1m2pgh6/data-grabber".
210522-20:57:57,536 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210522-20:57:57,540 nipype.workflow INFO:
	 [Node] Finished "binarise.data-grabber".
210522-20:57:57,541 nipype.workflow INFO:
	 [Node] Setting-up "binarise.apply-bin" in "/tmp/tmprhc4j0w8/binarise/_TR_1510_sub_03/apply-bin".
210522-20:57:57,547 nipype.workflow INFO:
	 [Node] Setting-up "_apply-bin0" in "/tmp/tmprhc4j0w8/binarise/_TR_1510_sub_03/apply-bin/mapflow/_apply-bin0".
210522-20:57:57,551 nipype.workflow INFO:
	 [Node] Running "_apply-bin0" ("nipype.interfaces.fsl.maths.UnaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-03/TR1510/THRSH/spmT_0004_thresh.nii.gz -nan -bin -thr .01 /tmp/tmprhc4j0w8/binar

210522-20:57:58,614 nipype.workflow INFO:
	 [Node] Setting-up "_apply-bin1" in "/tmp/tmpg34it90_/binarise/_TR_1920_sub_02/apply-bin/mapflow/_apply-bin1".
210522-20:57:58,619 nipype.workflow INFO:
	 [Node] Running "_apply-bin1" ("nipype.interfaces.fsl.maths.UnaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-02/TR1920/THRSH/spmT_0005_thresh.nii.gz -nan -bin -thr .01 /tmp/tmpg34it90_/binarise/_TR_1920_sub_02/apply-bin/mapflow/_apply-bin1/spmT_0005_thresh_bin.nii.gz
210522-20:57:58,763 nipype.workflow INFO:
	 [Node] Finished "_apply-bin1".
210522-20:57:58,765 nipype.workflow INFO:
	 [Node] Setting-up "_apply-bin2" in "/tmp/tmpg34it90_/binarise/_TR_1920_sub_02/apply-bin/mapflow/_apply-bin2".
210522-20:57:58,770 nipype.workflow INFO:
	 [Node] Running "_apply-bin2" ("nipype.interfaces.fsl.maths.UnaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-02/TR1920/TH

210522-20:57:59,835 nipype.workflow INFO:
	 [Node] Finished "_apply-bin2".
210522-20:57:59,839 nipype.workflow INFO:
	 [Node] Finished "binarise.apply-bin".
210522-20:57:59,840 nipype.workflow INFO:
	 [Node] Setting-up "binarise.sink-stuff" in "/tmp/tmph_4mhc2b/binarise/_TR_700_sub_02/sink-stuff".
210522-20:57:59,848 nipype.workflow INFO:
	 [Node] Running "sink-stuff" ("nipype.interfaces.io.DataSink")
210522-20:57:59,849 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-02/TR700/MASK/_TR_700_sub_02/_apply-bin0/spmT_0004_thresh_bin.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-02/TR700/MASK///spmT_0004_thresh_bin.nii.gz
210522-20:57:59,851 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-02/TR700/MASK/_TR_700_sub_02/_apply-bin1/spmT_0005_thresh_bin.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-02/TR700/MASK///spmT_0005_thresh_bin.nii.gz
210522-20:57:59,853 nipype.

210522-20:58:00,961 nipype.workflow INFO:
	 [Node] Finished "binarise.sink-stuff".
210522-20:58:00,961 nipype.workflow INFO:
	 [Node] Setting-up "binarise.data-grabber" in "/tmp/tmpi1m2pgh6/data-grabber".
210522-20:58:00,966 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210522-20:58:00,970 nipype.workflow INFO:
	 [Node] Finished "binarise.data-grabber".
210522-20:58:00,970 nipype.workflow INFO:
	 [Node] Setting-up "binarise.apply-bin" in "/tmp/tmpim85lat2/binarise/_TR_700_sub_01/apply-bin".
210522-20:58:00,978 nipype.workflow INFO:
	 [Node] Setting-up "_apply-bin0" in "/tmp/tmpim85lat2/binarise/_TR_700_sub_01/apply-bin/mapflow/_apply-bin0".
210522-20:58:00,983 nipype.workflow INFO:
	 [Node] Running "_apply-bin0" ("nipype.interfaces.fsl.maths.UnaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_TC/sub-01/TR700/THRSH/spmT_0004_thresh.nii.gz -nan -bin -thr .01 /tmp/tmpim85lat2/binarise